In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


2025-04-17 17:56:50.698414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744912611.026073      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744912611.119220      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Update these paths if needed
dataset1_train = '/kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train'
dataset1_test = '/kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_test'
dataset2_train = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
dataset2_test = '/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'

# If you want to merge both datasets, you can create a list of directories
train_dirs = [dataset1_train, dataset2_train]
test_dirs = [dataset1_test, dataset2_test]

train_dirs

['/kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train',
 '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train']

In [3]:
IMG_SIZE = 96
BATCH_SIZE = 64

# Combine all train and test directories
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Helper function to create generators for multiple directories
def create_generator(dirs, datagen, subset=None, shuffle=True):
    generators = []
    for d in dirs:
        if os.path.exists(d):
            gen = datagen.flow_from_directory(
                d,
                target_size=(IMG_SIZE, IMG_SIZE),
                batch_size=BATCH_SIZE,
                class_mode='categorical',
                subset=subset,
                shuffle=shuffle
            )
            generators.append(gen)
    return generators

train_gens = create_generator(train_dirs, train_datagen, subset='training')
val_gens = create_generator(train_dirs, train_datagen, subset='validation')
# test_gens = create_generator(test_dirs, test_datagen, shuffle=False)

# Optionally, you can concatenate the generators or use only one dataset if merging is complex
train_gen = train_gens[0]  # or use a custom generator to merge batches
val_gen = val_gens[0]
# test_gen = test_gens[0]


Found 200779 images belonging to 29 classes.
Found 78300 images belonging to 29 classes.
Found 22295 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [4]:
print("Number of training images:", train_gen.samples)
print("Number of classes:", train_gen.num_classes)
print("Class indices:", train_gen.class_indices)

Number of training images: 200779
Number of classes: 29
Class indices: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_SIZE = 96

model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(29, activation='softmax')
])

# Custom learning rate for Adam optimizer
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary to check parameter count
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1744912781.176453      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744912781.177165      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 94, 94, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 47, 47, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 45, 45, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 22, 22, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 20, 20, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 8, 8, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │          32,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 29)                  │             957 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 94,269 (368.24 KB)

 Trainable params: 94,269 (368.24 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
EPOCHS = 30

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
)


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1744912789.102804      89 service.cc:148] XLA service 0x7a6b40050070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744912789.103862      89 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1744912789.103889      89 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1744912789.486859      89 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/3138 ━━━━━━━━━━━━━━━━━━━━ 7:46:39 9s/step - accuracy: 0.0312 - loss: 3.3749

I0000 00:00:1744912792.514410      89 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3138/3138 ━━━━━━━━━━━━━━━━━━━━ 1970s 625ms/step - accuracy: 0.1012 - loss: 3.1282 - val_accuracy: 0.2788 - val_loss: 2.9392
Epoch 2/30
3138/3138 ━━━━━━━━━━━━━━━━━━━━ 1191s 379ms/step - accuracy: 0.3708 - loss: 2.1137 - val_accuracy: 0.3582 - val_loss: 2.9230
Epoch 3/30
3138/3138 ━━━━━━━━━━━━━━━━━━━━ 1122s 357ms/step - accuracy: 0.4874 - loss: 1.7066 - val_accuracy: 0.4082 - val_loss: 2.8782
Epoch 4/30
3138/3138 ━━━━━━━━━━━━━━━━━━━━ 1149s 366ms/step - accuracy: 0.5610 - loss: 1.4507 - val_accuracy: 0.4315 - val_loss: 2.7972
Epoch 5/30
3138/3138 ━━━━━━━━━━━━━━━━━━━━ 1372s 437ms/step - accuracy: 0.6172 - loss: 1.2583 - val_accuracy: 0.4614 - val_loss: 2.7063
Epoch 6/30
3138/3138 ━━━━━━━━━━━━━━━━━━━━ 3328s 1s/step - accuracy: 0.6631 - loss: 1.1075 - val_accuracy: 0.4827 - val_loss: 2.6920
Epoch 7/30
3138/3138 ━━━━━━━━━━━━━━━━━━━━ 1053s 335ms/step - accuracy: 0.6956 - loss: 0.9882 - val_accuracy: 0.5006 - val_loss: 2.6679
Epoch 8/30
3138/3138 ━━━━━━━━━━━━━━━━━━━━ 1086s 345ms/step - accuracy

In [7]:
# Save the model in the native Keras format (.keras)
model.save("asl_cnn_model4.keras")